# Practice Association Rules  - Minneapolis Crime Data

The dataset used in the notebook (curtesy of Open Data Minneapolis) includes information about police calls and crimes committed between 2010 to 2016. 

We will use the data to do some association rule mining for finding frequent patterns. More specifically we will explore association between crime type, location, and time. 

Read the data from `/dsa/data/all_datasets/minneapolis_crimedata/crimes.csv`


In [ ]:
import pandas as pd
import datetime
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
crimes_data = pd.read_csv('/dsa/data/all_datasets/minneapolis_crimedata/crimes.csv')

In [ ]:
crimes_data.head()

In [ ]:
crimes_data.shape

The columns 
- `controlnbr`
- `CCN`
- `Time`
- `ReportedDate`
- `Offense`
- `UCRCode`
- `EnteredDate`
- `Long`
- `Lat`
- `x`
- `y`
- `lastchanged`
- `LastUpdateDate`
- `OBJECTID`
- `ESRI_OID` 

are not helpful or interpretable. So lets just delete them from dataset.

**Activity 1:** publicaddress, BeginDate, and Description. 

In [ ]:
# Your code for activity 1 goes here..
crimes_data = crimes_data.iloc[:, [0,5,8]]

In [ ]:
crimes_data.head()

The first 6 characters in publicaddress don't make any sense. 

**Activity 2:** Strip the first 7 characters or extract the rest of the characters from the publicaddress column.

In [ ]:
crimes_data['publicaddress'] = crimes_data['publicaddress'].str[7:]

In [ ]:
crimes_data.head()




**Activity 3:** Convert BeginDate to date_time using pandas to_datetime. Add two new columns by extracting weekday and hour from the BeginDate. 

In [ ]:
crimes_data['BeginDate'] = pd.to_datetime(crimes_data['BeginDate'],utc=True)
crimes_data['BeginDate'] = pd.to_datetime(crimes_data['BeginDate'],errors='coerce').dt.tz_localize(None)

In [ ]:
crimes_data.head()

In [ ]:
#crimes_data['date'] = (crimes_data.BeginDate).dt.date
crimes_data['weekday'] = (crimes_data.BeginDate).dt.day_name()
crimes_data['hour'] = (crimes_data.BeginDate).dt.hour

In [ ]:
crimes_data.head()

**Activity 6:** Convert the hour variable into an ordered factor with levels "mid night", "morning", "noon","night" for different hours of the day. 

In [ ]:
# In this example we will use pd.cut() and bins to create the new column with grouping
bins= [0, 4, 11,14, 24]
labels = ["mid night", "morning", "noon","night"]
crimes_data['hour'] = pd.cut(crimes_data['hour'], bins=bins, labels=labels, right=False)

In [ ]:
crimes_data.head()

We dont need the BeginDate column any more. So lets just delete it from dataframe.

In [ ]:
crimes_data = crimes_data.drop(['BeginDate'],axis=1)

In [ ]:
crimes_data.head()

**Activity 8:** Now, coerce the data set into transactions. Save this transactions to crimes_trans variable.

Hint: Use `get_dummies` function from pandas. see [here](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html)

In [ ]:
crimes_data = pd.get_dummies(crimes_data.astype(str))

In [ ]:
crimes_data.head()

In [ ]:
crimes_data.shape

**Activity 9:** Slice the first 10000 transactions into a new dataframe. The new transaction data is too large for us to compute in this server, so we do for the first 10000 rows.

In [ ]:
crimes_trans = crimes_data[:10000]

**Activity 10:** Generate association rules for the transactions in crimes_trans with support of 0.01 and confidence of 0.5

In [ ]:
# step 1: identify frequent itemsets
frequent_itemsets = apriori(crimes_trans, min_support = 0.01,use_colnames=True)

# step 2: infer rules frequent itemsets
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)


In [ ]:
print(f"Number of rules = {rules.shape[0]}")

**Activity 11:** Display top-5 generated rules in terms of conviction. 

In [ ]:
rules_sorted = rules.sort_values(by =['conviction'], ascending=False)

In [ ]:
rules_sorted.head()

**Activity 12:** Show some rules that are surprising. Lift and leverage are used to identify surprising rules. 

In [ ]:
plt.scatter(rules['lift'], rules['leverage'], alpha=0.5)
plt.xlabel('lift')
plt.ylabel('leverage')
plt.title('Lift vs Leverage');

In [ ]:
rules_sub = rules[(rules['lift'] >= 1.6)]
rules_sub.head()

In [ ]:
rules_sub = rules[(rules['leverage'] >= 0.01)]
rules_sub.head()

# Save your notebook, then `File > Close and Halt`